In [1]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
!pip install keras 
from tensorflow.keras import models,layers
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.applications import *
from tensorflow.keras.preprocessing import image

In [2]:
batch = 32
classes = 'binary'

train='C:/Users/Shusmita/Desktop/Oral Cancer/Augmented_Oral_Cancer/train'
test='C:/Users/Shusmita\Desktop\Oral Cancer/Augmented_Oral_Cancer/test'
val='C:/Users/Shusmita/Desktop/Oral Cancer/Augmented_Oral_Cancer/Validate'

In [3]:
train_datagen = image.ImageDataGenerator(
    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)
validation_datagen= image.ImageDataGenerator()

test_datagen= image.ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    train,
    target_size = (224,224),
    batch_size = batch,
    class_mode = classes)


validation_generator = validation_datagen.flow_from_directory(
    val,
    target_size = (224,224),
    batch_size = batch,
    shuffle=True,
    class_mode = classes)

test_generator = test_datagen.flow_from_directory(
    test,
    target_size = (224,224),
    batch_size = batch,
    class_mode = classes)
class_names=validation_generator.class_indices
class_names
len(class_names)

Found 619 images belonging to 2 classes.
Found 130 images belonging to 2 classes.
Found 140 images belonging to 2 classes.


2

In [4]:
base_for_model = resnet.ResNet101(weights='imagenet', input_shape=(224,224,3), include_top=False)
for layer in base_for_model.layers:
    layer.trainable = False

A local file was found, but it seems to be incomplete or outdated because the auto file hash does not match the original value of 88cf7a10940856eca736dc7b7e228a21 so we will re-download the data.
171446536/171446536 [==============================] - 396s 2us/step


In [5]:
model = Sequential()
model.add(base_for_model) 
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(512, activation='relu'))
model.add(GlobalMaxPooling2D()) 
model.add(BatchNormalization())
model.add(Dense(512, activation = 'relu')) 
model.add(BatchNormalization()) 
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet101 (Functional)      (None, 7, 7, 2048)        42658176  
                                                                 
 dense (Dense)               (None, 7, 7, 512)         1049088   
                                                                 
 batch_normalization (BatchN  (None, 7, 7, 512)        2048      
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 7, 7, 512)         262656    
                                                                 
 global_max_pooling2d (Globa  (None, 512)              0         
 lMaxPooling2D)                                                  
                                                                 
 batch_normalization_1 (Batc  (None, 512)              2

In [6]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=['accuracy','Precision','Recall','AUC']
)

In [7]:
history=model.fit(
    train_generator,
    epochs=10,
    batch_size=64,
    validation_data=validation_generator
)

Epoch 1/10
20/20 [==============================] - 133s 6s/step - loss: 0.4700 - accuracy: 0.8304 - precision: 0.8239 - recall: 0.8424 - auc: 0.9063 - val_loss: 0.4239 - val_accuracy: 0.8231 - val_precision: 0.7470 - val_recall: 0.9688 - val_auc: 0.9444
Epoch 2/10
20/20 [==============================] - 129s 7s/step - loss: 0.2323 - accuracy: 0.9192 - precision: 0.9365 - recall: 0.9003 - auc: 0.9767 - val_loss: 0.5125 - val_accuracy: 0.8769 - val_precision: 0.9800 - val_recall: 0.7656 - val_auc: 0.9640
Epoch 3/10
20/20 [==============================] - 130s 7s/step - loss: 0.1326 - accuracy: 0.9499 - precision: 0.9459 - recall: 0.9550 - auc: 0.9889 - val_loss: 0.8892 - val_accuracy: 0.8462 - val_precision: 0.9783 - val_recall: 0.7031 - val_auc: 0.9303
Epoch 4/10
20/20 [==============================] - 128s 6s/step - loss: 0.0855 - accuracy: 0.9725 - precision: 0.9804 - recall: 0.9646 - auc: 0.9923 - val_loss: 0.3794 - val_accuracy: 0.9000 - val_precision: 1.0000 - val_recall: 0.796

In [8]:
score=model.evaluate(validation_generator)
score

5/5 [==============================] - 22s 4s/step - loss: 0.0988 - accuracy: 0.9538 - precision: 1.0000 - recall: 0.9062 - auc: 0.9998


[0.09878158569335938, 0.9538461565971375, 1.0, 0.90625, 0.9997632503509521]